<a href="https://colab.research.google.com/github/shiodeaiko/shiodeaiko/blob/main/nennkinn_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

元のコード


In [201]:
#必要なライブラリのインポート
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

#アニメーション
from matplotlib.animation import PillowWriter, FuncAnimation
from matplotlib import animation, rc
from IPython.display import HTML

#Scipy
from scipy import sparse
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix

## 5個のノードからなる無向グラフ


In [202]:
#グラフの生成
G=nx.Graph()
G.add_nodes_from([0,1,2,3,4])
G.add_edges_from([(0,2),(0,3),(2,3),(2,4),(1,4),(1,3),(3,4)])
pos = nx.spring_layout(G)

In [203]:
#変数宣言
nt=13    #17以上はLinAlgError: Singular matrix
#今回はdt=1とする
nnodes=nx.number_of_nodes(G)
edge_list=list(G.edges)

#隣接行列の生成
nbr_mtx=nx.to_numpy_matrix(G)

conductivity=np.zeros((nt,nnodes,nnodes)) #D
length=np.zeros((nnodes,nnodes)) #L
pressure=np.zeros((nt,nnodes)) #p
flux=np.zeros(nnodes) #Q


In [204]:
#初期値と定数の設定
conductivity[0]=nbr_mtx #各エッジの流れやすさは最初1.0
length=nbr_mtx #今回は各エッジの長さは1.0とする

I0=2.0
flux[0]=I0 #流入口
flux[1]=-I0 #流出口

gamma=1.8

In [205]:
#ゼロ除算が発生しないように非ゼロ要素のみで割り算を行う
def divide_non_zero_element(D,L,num_nodes,list_edges):
    X=np.zeros((num_nodes,num_nodes))
    for i,j in list_edges:
        X[i,j]=D[i,j]/L[i,j]
        X[j,i]=D[j,i]/L[j,i]
    return X

#f(Q) for dD/dt 
def f(x):
    powered=x**gamma
    return powered/(powered+1)

#Dの時間変化量を求める
def dD(D,L,p,num_nodes,list_edges):
    X=divide_non_zero_element(D,L,num_nodes,list_edges)
    Q=np.multiply(X,np.expand_dims(p,axis=1)-p)         # np.multiply() 行列同士の要素ごとの積
                                                        # np.expand_dims(元の配列,axis=追加する位置) 新たな次元の追加
    ans=f(np.abs(Q))  # np.abs() 絶対値
    return ans

#一次連立方程式を解きpを求める
def deduce_p(D,L,B,num_nodes,list_edges):
    Y=divide_non_zero_element(D,L,num_nodes,list_edges)    
    A=np.diag(np.sum(Y,axis=1))-Y   # np.diag() 対角成分を抽出
                                     # sum() axis 足し合わせる方向 0;行方向 1;列方向
    p=np.linalg.solve(A,B)
    return p

In [206]:
#pの初期値を求める
pressure[0]=deduce_p(conductivity[0],length,flux,nnodes,edge_list)

#繰り返し計算する
for t in range(0,nt-1):
    conductivity[t+1]=dD(conductivity[t],length,pressure[t],nnodes,edge_list)
    pressure[t+1]=deduce_p(conductivity[t+1],length,flux,nnodes,edge_list)

In [207]:
#%matplotlib nbagg
fig=plt.figure()

datatype=[('conductivity',float)]

def animate(i):
    A=np.matrix(conductivity[i],dtype=datatype)
    G=nx.from_numpy_matrix(A)
    weights=[G[u][v]['conductivity'] for u,v in G.edges()]

    plt.cla()
    nx.draw_networkx(G,pos,width=weights)
    plt.axis('off')
    plt.title('t=' + str(i))
    #plt.show()

# アニメーションの設定
ani = animation.FuncAnimation(fig,animate,frames=nt,repeat=True)

# colabでアニメーションを動かすためのコード
rc('animation', html='jshtml')
ani

<IPython.core.display.Javascript object>

## より複雑なグラフでの試行

In [208]:
#グラフの生成
G0=nx.triangular_lattice_graph(10,10)
pos0=nx.spring_layout(G0)
nx.draw_networkx(G0,pos0)

#ノードのラベルを1次に変更
nbr_mtx=nx.to_numpy_matrix(G0)
G=nx.from_numpy_matrix(nbr_mtx)
pos=nx.spring_layout(G,iterations=100)
nx.draw_networkx(G,pos)

In [209]:
#変数宣言

nt=10

#dt=1とする
nnodes=nx.number_of_nodes(G)
edge_list=list(G.edges)
nbr_mtx=nx.to_numpy_matrix(G)

conductivity=np.zeros((nt,nnodes,nnodes)) #D
length=np.zeros((nnodes,nnodes)) #L
pressure=np.zeros((nt,nnodes)) #p
flux=np.zeros(nnodes) #Q

In [210]:
#初期値と定数の設定
conductivity[0]=nbr_mtx #各エッジの流れやすさは最初1.0
length=nbr_mtx #今回は各エッジの長さは1.0とする
I0=2.0

source_list=[0,65] #流入口
sink_list=[30] #流出口
source_len=len(source_list)
sink_len=len(sink_list)

for i in source_list:
    flux[i]=I0

#餌ノードの量が全体で保存されるように調整する
for i in sink_list:
    flux[i]=-I0*(source_len/sink_len)

gamma=1.5

In [211]:
#ゼロ除算が発生しないように非ゼロ要素のみで割り算を行う
def divide_non_zero_element(D,L,num_nodes,list_edges):
    X=np.zeros((num_nodes,num_nodes))
    for i,j in list_edges:
        X[i,j]=D[i,j]/L[i,j]
        X[j,i]=D[j,i]/L[j,i]
    return X

#f(Q) for dD/dt 
def f(x):
    powered=x**gamma
    return powered/(powered+1)

#Dの時間変化量を求める
def dD(D,L,p,num_nodes,list_edges):
    X=divide_non_zero_element(D,L,num_nodes,list_edges)
    Q=np.multiply(X,np.expand_dims(p,axis=1)-p)
    ans=f(np.abs(Q))
    return ans

#一次連立方程式を解きpを求める
def deduce_p(D,L,B,num_nodes,list_edges):
    Y=divide_non_zero_element(D,L,num_nodes,list_edges)    
    A=np.diag(np.sum(Y,axis=1))-Y
    p=np.linalg.solve(A,B)
    #
    return p

In [212]:
#pの初期値を求める
pressure[0]=deduce_p(conductivity[0],length,flux,nnodes,edge_list)

#繰り返し計算する
for t in range(0,nt-1):
    conductivity[t+1]=dD(conductivity[t],length,pressure[t],nnodes,edge_list)
    pressure[t+1]=deduce_p(conductivity[t+1],length,flux,nnodes,edge_list)

In [213]:
%matplotlib nbagg
fig=plt.figure()

datatype=[('conductivity',float)]

def animate(i):
    A=np.matrix(conductivity[i],dtype=datatype)
    G=nx.from_numpy_matrix(A)
    weights=[G[u][v]['conductivity'] for u,v in G.edges()]

    plt.cla()
    nx.draw_networkx(G,pos,width=weights,node_size=20)
    nx.draw_networkx_nodes(G,pos,node_size=50,node_color=pressure[i],cmap=plt.cm.brg)
    plt.axis('off')
    plt.title('t=' + str(i))
    #plt.show()


# アニメーションの設定
ani = animation.FuncAnimation(fig,animate,frames=nt,repeat=True)

# colabでアニメーションを動かすためのコード
rc('animation', html='jshtml')
ani

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/cbook/__init__.py", line 196, in process
    func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/animation.py", line 1467, in _stop
    self.event_source.remove_callback(self._loop_delay)
AttributeError: 'NoneType' object has no attribute 'remove_callback'
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/cbook/__init__.py", line 196, in process
    func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/animation.py", line 1467, in _stop
    self.event_source.remove_callback(self._loop_delay)
AttributeError: 'NoneType' object has no attribute 'remove_callback'
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/cbook/__init__.py", line 196, in process
    func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/matplotlib/animation.py", line 1467, 

<IPython.core.display.Javascript object>